In [15]:
import numpy as np

N = 100 # number of samples
L = 1000 # length of each sample (number of values for each sine wave)
T = 20 # width of the wave
x = np.empty((N,L), np.float32) # instantiate empty array
x[:] = np.arange(L) + np.random.randint(-4*T, 4*T, N).reshape(N,1)
y = np.sin(x/1.0/T).astype(np.float32)

In [16]:
import torch #pytorch
import torch.nn as nn


class LSTM(nn.Module):
    def __init__(self, n_hidden=64):
        super(LSTM, self).__init__()
        self.n_hidden =n_hidden
        # lstm1, lstm2, linear are all layers in the network
        self.lstm1 = nn.LSTMCell(1, self.n_hidden)
        self.lstm2 = nn.LSTMCell(self.n_hidden, self.n_hidden)
        self.linear = nn.Linear(self.n_hidden, 1)
        
    def forward(self, x, future=0):
        outputs, n_samples = [], x.size(0)
        h_t = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32)
        c_t = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32)
        h_t2 = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32)
        c_t2 = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32)
        
        for input_t in x.split(1, dim=1):
            # N, 1
            h_t, c_t = self.lstm1(input_t, (h_t, c_t)) # initial hidden and cell states
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2)) # new hidden and cell states
            output = self.linear(h_t2) # output from the last FC layer
            outputs.append(output)
            
        for i in range(future):
            # this only generates future predictions if we pass in future_preds>0
            # mirrors the code above, using last output/prediction as input
            h_t, c_t = self.lstm1(output, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs.append(output)
        # transform list to tensor    
        outputs = torch.cat(outputs, dim=1)
        return outputs


In [17]:
import matplotlib.pyplot as plt
train_input = torch.from_numpy(y[3:, :-1])  # (97, 999)
train_target = torch.from_numpy(y[3:, 1:])  # (97, 999)
test_input = torch.from_numpy(y[:3, :-1])  # (3, 999)
test_target = torch.from_numpy(y[:3, 1:])  # (3, 999)

model = LSTM()
criterion = nn.MSELoss()
optimiser = torch.optim.LBFGS(model.parameters(), lr=0.2)
n_epochs = 5

for i in range(n_epochs):
    def closure():
        optimiser.zero_grad()
        out = model(train_input)
        loss = criterion(out, train_target)
        print(f"Learning Loss {loss.item()}")
        loss.backward()
        return loss
    optimiser.step(closure)
    with torch.no_grad():
        future = 1000
        pred = model(test_input, future=future)
        # use all pred samples, but only go to 999
        loss = criterion(pred[:, :-future], test_target)
        print("test loss", loss.item())
        y = pred.detach().numpy()
    # draw figures
    plt.figure(figsize=(12, 6))
    plt.title(f"Step {i+1}")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    n = train_input.shape[1]  # 999

    def draw(yi, colour):
        plt.plot(np.arange(n), yi[:n], colour, linewidth=2.0)
        plt.plot(np.arange(n, n + future), yi[n:], colour + ":", linewidth=2.0)
    draw(y[0], 'r')
    draw(y[1], 'b')
    draw(y[2], 'g')
    plt.savefig("predict%d.png" % i, dpi=200)
    plt.close()
    # print the loss
    out = model(train_input)
    loss_print = criterion(out, train_target)
    print("Step: {}, Loss: {}".format(i, loss_print))


Learning Loss 0.5132271647453308
Learning Loss 0.5121118426322937
Learning Loss 0.505466103553772
Learning Loss 0.49633949995040894
Learning Loss 0.48312634229660034
Learning Loss 0.46188783645629883
Learning Loss 0.4206937253475189
Learning Loss 0.30492761731147766
Learning Loss 0.2609255015850067
Learning Loss 0.21976105868816376
Learning Loss 0.12627197802066803
Learning Loss 0.06313163042068481
Learning Loss 0.04263348504900932
Learning Loss 0.032605938613414764
Learning Loss 0.026671530678868294
Learning Loss 0.022975996136665344
Learning Loss 0.020613977685570717
Learning Loss 0.019068419933319092
Learning Loss 0.018028946593403816
Learning Loss 0.017304640263319016
test loss 0.016174130141735077
Step: 0, Loss: 0.01677592471241951
Learning Loss 0.01677592471241951
Learning Loss 0.016366416588425636
Learning Loss 0.016025805845856667
Learning Loss 0.015719154849648476
Learning Loss 0.015420042909681797
Learning Loss 0.015106109902262688
Learning Loss 0.014756185933947563
Learning 

In [13]:
print(test_input.split(1, dim=1))

(tensor([[-0.4425],
        [-0.5155],
        [-0.2555]]), tensor([[-0.4868],
        [-0.5577],
        [-0.3035]]), tensor([[-0.5298],
        [-0.5985],
        [-0.3508]]), tensor([[-0.5716],
        [-0.6378],
        [-0.3971]]), tensor([[-0.6119],
        [-0.6755],
        [-0.4425]]), tensor([[-0.6506],
        [-0.7115],
        [-0.4868]]), tensor([[-0.6878],
        [-0.7457],
        [-0.5298]]), tensor([[-0.7232],
        [-0.7781],
        [-0.5716]]), tensor([[-0.7568],
        [-0.8085],
        [-0.6119]]), tensor([[-0.7885],
        [-0.8369],
        [-0.6506]]), tensor([[-0.8183],
        [-0.8632],
        [-0.6878]]), tensor([[-0.8460],
        [-0.8874],
        [-0.7232]]), tensor([[-0.8716],
        [-0.9093],
        [-0.7568]]), tensor([[-0.8950],
        [-0.9290],
        [-0.7885]]), tensor([[-0.9162],
        [-0.9463],
        [-0.8183]]), tensor([[-0.9351],
        [-0.9613],
        [-0.8460]]), tensor([[-0.9516],
        [-0.9738],
        [-0.8716]